In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, when

spark = SparkSession.builder.appName("AmazonBooksEDA").getOrCreate()

# Carga de archivos
books_df = spark.read.option("header", True).option("inferSchema", True).csv("/app/data/raw/books_data.csv")
ratings_df = spark.read.option("header", True).option("inferSchema", True).csv("/app/data/raw/Books_rating.csv")



/usr/local/lib/python3.11/site-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/25 01:26:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
print("books_data.csv:")
books_df.show(5)
books_df.printSchema()

print("\nBooks_rating.csv:")
ratings_df.show(5)
ratings_df.printSchema()


books_data.csv:
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+------------+
|               Title|         description|             authors|               image|         previewLink|           publisher| publishedDate|            infoLink|          categories|ratingsCount|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+------------+
|Its Only Art If I...|                NULL|    ['Julie Strain']|http://books.goog...|http://books.goog...|                NULL|          1996|http://books.goog...|['Comics & Graphi...|        NULL|
|Dr. Seuss: Americ...|"Philip Nel takes...| like that of Lew...| has changed lang...| giving us new wo...| inspiring artist...|['Philip Nel']|http://books.goog...|http://books.goog...|   A&C B

In [3]:
print("books_data.csv - filas:", books_df.count(), "columnas:", len(books_df.columns))
print("Books_rating.csv - filas:", ratings_df.count(), "columnas:", len(ratings_df.columns))


books_data.csv - filas: 212404 columnas: 10


Books_rating.csv - filas: 3000000 columnas: 10


In [4]:
print("Valores nulos en books_data.csv:")
books_df.select([count(when(col(c).isNull(), c)).alias(c) for c in books_df.columns]).show()

print("Valores nulos en Books_rating.csv:")
ratings_df.select([count(when(col(c).isNull(), c)).alias(c) for c in ratings_df.columns]).show()



Valores nulos en books_data.csv:


+-----+-----------+-------+-----+-----------+---------+-------------+--------+----------+------------+
|Title|description|authors|image|previewLink|publisher|publishedDate|infoLink|categories|ratingsCount|
+-----+-----------+-------+-----+-----------+---------+-------------+--------+----------+------------+
|    1|      68357|  31251|51191|      24055|    73130|        25844|   24301|     40524|      148552|
+-----+-----------+-------+-----+-----------+---------+-------------+--------+----------+------------+

Valores nulos en Books_rating.csv:


+---+-----+-------+-------+-----------+------------------+------------+-----------+--------------+-----------+
| Id|Title|  Price|User_id|profileName|review/helpfulness|review/score|review/time|review/summary|review/text|
+---+-----+-------+-------+-----------+------------------+------------+-----------+--------------+-----------+
|  0|  208|2517579| 562250|     562200|               367|         130|         27|            65|         43|
+---+-----+-------+-------+-----------+------------------+------------+-----------+--------------+-----------+



In [5]:
print("Estadísticas books_data.csv:")
books_df.describe().show()

print("Ratings únicos en books_data.csv:")
books_df.select("ratingsCount").distinct().show()

print("Estadísticas Books_rating.csv:")
ratings_df.describe().show()


25/04/25 01:26:57 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Estadísticas books_data.csv:


+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|summary|               Title|         description|             authors|               image|         previewLink|           publisher|       publishedDate|            infoLink|          categories|        ratingsCount|
+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|  count|              212403|              144047|              181153|              161213|              188349|              139274|              186560|              188103|              171880|               63852|
|   mean|   3823.672941176471|  1.4285714285714286|              1578.4|              1184.0|            Infinity|      

+--------------------+
|        ratingsCount|
+--------------------+
|                 air|
|              Korean|
|Natural Law and E...|
|http://books.goog...|
|             Inc."""|
|               102.0|
| and the Universi...|
| and multiplexing...|
| service (or prod...|
|              Mathyz|
| make this sophis...|
|             au bout|
|          2014-05-27|
| Eureka! perfectl...|
| and paranormal n...|
| rest in its vill...|
| those who surviv...|
| Sharpe's Trafalg...|
|http://books.goog...|
| he escalates the...|
+--------------------+
only showing top 20 rows

Estadísticas Books_rating.csv:


+-------+--------------------+--------------------+--------------------+-------------------+-----------+-------------------+------------------+--------------------+--------------------+--------------------+
|summary|                  Id|               Title|               Price|            User_id|profileName| review/helpfulness|      review/score|         review/time|      review/summary|         review/text|
+-------+--------------------+--------------------+--------------------+-------------------+-----------+-------------------+------------------+--------------------+--------------------+--------------------+
|  count|             3000000|             2999792|              482421|            2437750|    2437800|            2999633|           2999870|             2999973|             2999935|             2999957|
|   mean|1.0568515696607149E9|   2012.796651763537|  21.767951161877054|  18.29299003322259|        NaN|3.285048033703448E8| 1656.860421970827|1.1270533345949814E9|        